In [18]:
import gc
import itertools
import os
import pickle
import random
import sys
import warnings
from glob import glob
from pathlib import Path

import config  # edit config.py as needed
import joblib
import lightgbm as lgb
import numpy as np
import pandas as pd
import polars as pl
import scipy as sp
import torch
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from lifelines import KaplanMeierFitter
from metric import score  # edit metric.py as needed
from scipy.stats import rankdata
from seed import seed_everything  # edit seed.py as needed
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")


In [2]:
# ====================================================
# Configurations
# ====================================================
class CFG:
    EXP_NAME = config.EXP_NAME
    AUTHOR = config.KAGGLE_USERNAME
    COMPETITION = config.KAGGLE_COMPETITION_NAME
    DATA_PATH = config.COMP_DATASET_DIR
    OUTPUT_DIR = config.OUTPUT_DIR
    MODEL_PATH = config.OUTPUT_DIR / "models"
    METHOD_LIST = ["lightgbm", "xgboost", "catboost"]
    SEED = 42
    n_folds = 5
    target_col_list = ["y"]
    # group_col = "race_group"  # Required for GroupKFold (edit as needed)
    # stratified_col = "race_group"  # Required for StratifiedKFold (edit as needed)
    stratified_cols = ["efs", "race_group"]  # Required for MultilabelStratifiedKFold（edit as needed）
    num_boost_round = 1000000
    early_stopping_round = 100
    verbose = 500

    # https://lightgbm.readthedocs.io/en/latest/Parameters.html
    regression_lgb_params = {
        "objective": "regression",
        "metric": "mae",
        "learning_rate": 0.02,
        # "num_leaves": 31,
        "max_depth": 3,
        "feature_fraction": 0.8,
        "seed": SEED,
        "device": "cpu",  # cpu/gpu/cuda
    }
    # https://xgboost.readthedocs.io/en/stable/parameter.html
    regression_xgb_params = {
        "objective": "reg:squarederror",
        "eval_metric": "mae",
        "learning_rate": 0.02,
        "max_depth": 3,
        "colsample_bytree": 0.8,
        "subsample": 0.8,
        "enable_categorical": True,
        "min_child_weight": 80,
        "random_state": SEED,
        "device": "cpu",  # cpu/gpu/cuda
    }
    # https://catboost.ai/docs/en/references/training-parameters/
    regression_cat_params = {
        "loss_function": "RMSE",
        "learning_rate": 0.02,
        "iterations": num_boost_round,
        # "depth": 7,
        "grow_policy": "Lossguide",
        "random_seed": SEED,
        "task_type": "CPU",  # CPU/GPU
    }

    model_weight_dict = {"lightgbm": 0.40, "xgboost": 0.30, "catboost": 0.30}


In [3]:
# ====================================================
# Seed everything
# ====================================================
seed_everything(CFG.SEED)


In [4]:
# ====================================================
# Read data
# ====================================================
train = pl.read_csv(CFG.DATA_PATH / "train.csv", try_parse_dates=True)
test = pl.read_csv(CFG.DATA_PATH / "test.csv", try_parse_dates=True)
sample_submission = pl.read_csv(CFG.DATA_PATH / "sample_submission.csv")
# make index column
# train = train.with_row_index()
# test = test.with_row_index()


In [5]:
# ====================================================
# Preprocess(ここに前処理や特徴量エンジニアリングを記述)
# ====================================================
def transform_survival_probability(df, time_col="efs_time", event_col="efs"):
    kmf = KaplanMeierFitter()
    kmf.fit(df[time_col], df[event_col])
    y = kmf.survival_function_at_times(df[time_col]).values
    return y


# def preprocess(df: pl.DataFrame) -> pl.DataFrame:
#     output = df.clone()
#     return output


In [6]:
# train = preprocess(train)
# test = preprocess(test)

# apply Kaplan-Meier
# TODO: もっと適切な目的変数があるかも（人種で層別化してモデル作るとか？）
y = transform_survival_probability(train, time_col="efs_time", event_col="efs")
train = train.with_columns(pl.Series(y).alias("y"))


In [7]:
# ====================================================
# Make fold column
# ====================================================
fold_array = np.zeros(len(train))

# skf = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.SEED)
# for i, (_, val_idx) in enumerate(skf.split(train, train[CFG.stratified_col])):
#     fold = i + 1
#     fold_array[val_idx] = fold
# train = train.with_columns(pl.Series(fold_array, dtype=pl.Int8).alias("fold"))

mskf = MultilabelStratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.SEED)
for i, (_, test_index) in enumerate(mskf.split(train, train[CFG.stratified_cols])):
    fold = i + 1
    fold_array[test_index] = fold
train = train.with_columns(pl.Series(fold_array, dtype=pl.Int8).alias("fold"))


In [8]:
train = train.to_pandas()
test = test.to_pandas()


In [9]:
# ====================================================
# Set categorical columns etc. (pandas operation from here)
# ====================================================
RMV = ["ID", "efs", "efs_time", "y", "fold"]
FEATURES = [c for c in train.columns if c not in RMV]
print(f"There are {len(FEATURES)} FEATURES: {FEATURES}")


There are 57 FEATURES: ['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_high', 'pulm_moderate', 'hla_low_res_10']


In [10]:
CATS = []
for c in FEATURES:
    if train[c].dtype == "object":
        CATS.append(c)
        train[c] = train[c].fillna("NAN")
        test[c] = test[c].fillna("NAN")
print(f"In these features, there are {len(CATS)} CATEGORICAL FEATURES: {CATS}")


In these features, there are 35 CATEGORICAL FEATURES: ['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status', 'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe', 'prior_tumor', 'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match', 'race_group', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'cardiac', 'pulm_moderate']


In [11]:
combined = pd.concat([train, test], axis=0, ignore_index=True)
# print("Combined data shape:", combined.shape )

# LABEL ENCODE CATEGORICAL FEATURES
print("We LABEL ENCODE the CATEGORICAL FEATURES: ", end="")
for c in FEATURES:
    # LABEL ENCODE CATEGORICAL AND CONVERT TO INT32 CATEGORY
    if c in CATS:
        print(f"{c}, ", end="")
        combined[c], _ = combined[c].factorize()
        combined[c] -= combined[c].min()
        combined[c] = combined[c].astype("int32")
        combined[c] = combined[c].astype("category")

    # REDUCE PRECISION OF NUMERICAL TO 32BIT TO SAVE MEMORY
    else:
        if combined[c].dtype == "float64":
            combined[c] = combined[c].astype("float32")
        if combined[c].dtype == "int64":
            combined[c] = combined[c].astype("int32")

train = combined.iloc[: len(train)].copy()
test = combined.iloc[len(train) :].reset_index(drop=True).copy()


We LABEL ENCODE the CATEGORICAL FEATURES: dri_score, psych_disturb, cyto_score, diabetes, tbi_status, arrhythmia, graft_type, vent_hist, renal_issue, pulm_severe, prim_disease_hct, cmv_status, tce_imm_match, rituximab, prod_type, cyto_score_detail, conditioning_intensity, ethnicity, obesity, mrd_hct, in_vivo_tcd, tce_match, hepatic_severe, prior_tumor, peptic_ulcer, gvhd_proph, rheum_issue, sex_match, race_group, hepatic_mild, tce_div_match, donor_related, melphalan_dose, cardiac, pulm_moderate, 

In [12]:
# ====================================================
# Training functions
# ====================================================
def lightgbm_training(
    x_train: pd.DataFrame,
    y_train: pd.DataFrame,
    x_valid: pd.DataFrame,
    y_valid: pd.DataFrame,
    # features: list,
    categorical_features: list,
):
    lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=categorical_features)
    lgb_valid = lgb.Dataset(x_valid, y_valid, categorical_feature=categorical_features)
    model = lgb.train(
        params=CFG.regression_lgb_params,
        train_set=lgb_train,
        num_boost_round=CFG.num_boost_round,
        valid_sets=[lgb_train, lgb_valid],
        callbacks=[
            lgb.early_stopping(stopping_rounds=CFG.early_stopping_round, verbose=CFG.verbose),
            lgb.log_evaluation(CFG.verbose),
        ],
    )
    # Predict validation
    valid_pred = model.predict(x_valid)
    return model, valid_pred


def xgboost_training(
    x_train: pd.DataFrame,
    y_train: pd.DataFrame,
    x_valid: pd.DataFrame,
    y_valid: pd.DataFrame,
    # features: list,
    # categorical_features: list,
):
    xgb_train = xgb.DMatrix(data=x_train, label=y_train, enable_categorical=True)
    xgb_valid = xgb.DMatrix(data=x_valid, label=y_valid, enable_categorical=True)
    model = xgb.train(
        CFG.regression_xgb_params,
        dtrain=xgb_train,
        num_boost_round=CFG.num_boost_round,
        evals=[(xgb_train, "train"), (xgb_valid, "eval")],
        early_stopping_rounds=CFG.early_stopping_round,
        verbose_eval=CFG.verbose,
    )
    # Predict validation
    valid_pred = model.predict(xgb.DMatrix(x_valid, enable_categorical=True))
    return model, valid_pred


def catboost_training(
    x_train: pd.DataFrame,
    y_train: pd.DataFrame,
    x_valid: pd.DataFrame,
    y_valid: pd.DataFrame,
    # features: list,
    categorical_features: list,
):
    cat_train = Pool(data=x_train, label=y_train, cat_features=categorical_features)
    cat_valid = Pool(data=x_valid, label=y_valid, cat_features=categorical_features)
    model = CatBoostRegressor(**CFG.regression_cat_params)
    model.fit(
        cat_train,
        eval_set=[cat_valid],
        early_stopping_rounds=CFG.early_stopping_round,
        verbose=CFG.verbose,
        use_best_model=True,
    )
    # Predict validation
    valid_pred = model.predict(x_valid)
    return model, valid_pred


def gradient_boosting_model_cv_training(
    method: str, train_df: pd.DataFrame, features: list, categorical_features: list
):
    # Create a numpy array to store out of folds predictions
    oof_predictions_df = pd.DataFrame(np.zeros((len(train_df), len(CFG.target_col_list))), columns=["prediction"])
    for target_col in CFG.target_col_list:
        oof_predictions = np.zeros(len(train_df))
        for fold in range(CFG.n_folds):
            print("-" * 50)
            print(f"{method} training fold {fold+1} {target_col}")
            x_train = train_df[train_df["fold"] != fold + 1][features]
            y_train = train_df[train_df["fold"] != fold + 1][target_col]
            x_valid = train_df[train_df["fold"] == fold + 1][features]
            y_valid = train_df[train_df["fold"] == fold + 1][target_col]
            if method == "lightgbm":
                model, valid_pred = lightgbm_training(x_train, y_train, x_valid, y_valid, categorical_features)
            if method == "xgboost":
                model, valid_pred = xgboost_training(x_train, y_train, x_valid, y_valid)
            if method == "catboost":
                model, valid_pred = catboost_training(x_train, y_train, x_valid, y_valid, categorical_features)

            # Save best model
            save_model_path = (
                CFG.MODEL_PATH / f"{method}_{target_col}_fold{fold + 1}_seed{CFG.SEED}_ver{CFG.EXP_NAME}.pkl"
            )
            save_model_path.parent.mkdir(parents=True, exist_ok=True)
            pickle.dump(
                model,
                open(
                    save_model_path,
                    "wb",
                ),
            )
            # Add to out of folds array
            oof_predictions[train_df["fold"] == fold + 1] = valid_pred
            del x_train, x_valid, y_train, y_valid, model, valid_pred
            gc.collect()
        # oof_predictions_df[target_col] = oof_predictions

    # Create a dataframe to store out of folds predictions
    oof_predictions_df["ID"] = train_df["ID"].values
    oof_predictions_df["prediction"] = oof_predictions
    oof_predictions_df.to_csv(CFG.OUTPUT_DIR / f"oof_{method}_seed{CFG.SEED}_ver{CFG.EXP_NAME}.csv", index=False)

    # Compute out of folds metric
    y_true = train_df[["ID", "efs", "efs_time", "race_group"]].copy()
    m = score(y_true.copy(), oof_predictions_df.copy(), "ID")
    print("=" * 50)
    print(f"{method} our out of folds CV score is {m}")
    print("=" * 50)


In [13]:
# ====================================================
# Training
# ====================================================
for method in CFG.METHOD_LIST:
    gradient_boosting_model_cv_training(method, train, features=FEATURES, categorical_features=CATS)


--------------------------------------------------
lightgbm training fold 1 y
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.745135 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 841
[LightGBM] [Info] Number of data points in the train set: 23040, number of used features: 57
[LightGBM] [Info] Start training from score 0.605593
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

In [19]:
# ====================================================
# Overall CV
# ====================================================
oof_lgb = (
    pl.read_csv(CFG.OUTPUT_DIR / f"oof_lightgbm_seed{CFG.SEED}_ver{CFG.EXP_NAME}.csv")
    .get_column("prediction")
    .to_numpy()
)
oof_xgb = (
    pl.read_csv(CFG.OUTPUT_DIR / f"oof_xgboost_seed{CFG.SEED}_ver{CFG.EXP_NAME}.csv")
    .get_column("prediction")
    .to_numpy()
)
oof_cat = (
    pl.read_csv(CFG.OUTPUT_DIR / f"oof_catboost_seed{CFG.SEED}_ver{CFG.EXP_NAME}.csv")
    .get_column("prediction")
    .to_numpy()
)

y_true = train[["ID", "efs", "efs_time", "race_group"]].copy()
y_pred = train[["ID"]].copy()
y_pred["prediction"] = rankdata(oof_xgb) + rankdata(oof_cat) + rankdata(oof_lgb)
m = score(y_true.copy(), y_pred.copy(), "ID")
print("\nOverall CV for Ensemble =", m)



Overall CV for Ensemble = 0.6739928287011623


In [20]:
# ====================================================
# Inference functions
# ====================================================
def lightgbm_inference(x_test: pd.DataFrame, target_col: str):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(
            open(
                CFG.MODEL_PATH / f"lightgbm_{target_col}_fold{fold + 1}_seed{CFG.SEED}_ver{CFG.EXP_NAME}.pkl",
                "rb",
            )
        )
        # Predict
        pred = model.predict(x_test)
        test_pred += pred
    return test_pred / CFG.n_folds


def xgboost_inference(x_test: pd.DataFrame, target_col: str):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(
            open(
                CFG.MODEL_PATH / f"xgboost_{target_col}_fold{fold + 1}_seed{CFG.SEED}_ver{CFG.EXP_NAME}.pkl",
                "rb",
            )
        )
        # Predict
        pred = model.predict(xgb.DMatrix(x_test, enable_categorical=True))
        test_pred += pred
    return test_pred / CFG.n_folds


def catboost_inference(x_test: pd.DataFrame, target_col: str):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(
            open(
                CFG.MODEL_PATH / f"catboost_{target_col}_fold{fold + 1}_seed{CFG.SEED}_ver{CFG.EXP_NAME}.pkl",
                "rb",
            )
        )
        # Predict
        pred = model.predict(x_test)
        test_pred += pred
    return test_pred / CFG.n_folds


def gradient_boosting_model_inference(method: str, test_df: pd.DataFrame, features: list, target_col: str):
    x_test = test_df[features]
    if method == "lightgbm":
        test_pred = lightgbm_inference(x_test, target_col)
    if method == "xgboost":
        test_pred = xgboost_inference(x_test, target_col)
    if method == "catboost":
        test_pred = catboost_inference(x_test, target_col)
    return test_pred


def predicting(input_df: pd.DataFrame, features: list):
    output_df = input_df.copy()
    for target_col in CFG.target_col_list:
        # output_df[target_col] = 0
        for method in CFG.METHOD_LIST:
            output_df[f"{method}_pred_{target_col}"] = gradient_boosting_model_inference(
                method, input_df, features, target_col
            )
            # output_df[target_col] += CFG.model_weight_dict[method] * output_df[f"{method}_pred_{target_col}"]
    return output_df


In [22]:
# ====================================================
# Inference
# ====================================================
output_df = predicting(test, FEATURES)
pred_lgb = output_df["lightgbm_pred_y"]
pred_xgb = output_df["xgboost_pred_y"]
pred_cat = output_df["catboost_pred_y"]

submission = pd.read_csv(CFG.DATA_PATH / "sample_submission.csv")
submission["prediction"] = rankdata(pred_lgb) + rankdata(pred_xgb) + rankdata(pred_cat)
submission.to_csv(CFG.OUTPUT_DIR / "submission.csv", index=False)
print("Sub shape:", submission.shape)
submission.head()


Sub shape: (3, 2)


,ID,prediction
0,28800,6.0
1,28801,9.0
2,28802,3.0
